<a href="https://www.kaggle.com/code/pes1ug22am164/image-part?scriptVersionId=171996997" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# PRE PROCESS
.JPG TO .NPY

In [1]:
# import os
# import cv2
# import numpy as np

# def jpg_to_npy_in_folders(input_folder, output_folder):
#     os.makedirs(output_folder, exist_ok=True)
#     for item in os.listdir(input_folder):
#         item_path = os.path.join(input_folder, item)
#         if os.path.isdir(item_path):
#             jpg_to_npy_in_folders(item_path, output_folder)
#         elif os.path.isfile(item_path) and item.endswith(".jpg"):
#             img = cv2.imread(item_path)
#             img_array = np.array(img)
#             output_file_path = os.path.join(output_folder, os.path.splitext(item)[0] + '.npy')
#             np.save(output_file_path, img_array)

# input_folder_path = "/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train"
# output_folder_path = "/kaggle/working/train"

# jpg_to_npy_in_folders(input_folder_path, output_folder_path)
# jpg_to_npy_in_folders("/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test", "/kaggle/working/test")



In [2]:
# import os
# import numpy as np
# directory = '/kaggle/input/betterer-preproc/archive/train'

# files = [file for file in os.listdir(directory) if file.endswith('.npy')]
# len(files)


In [3]:
# mildDem = [i for i in files if "mildDem" in i]
# moderateDem = [i for i in files if "moderateDem" in i]
# nonDem = [i for i in files if "nonDem" in i]
# veryMild = [i for i in files if "verymildDem" in i]
# print(len(mildDem), len(moderateDem), len(nonDem), len(veryMild))

In [4]:
# import imgaug.augmenters as iaa

# # Define the augmenter
# augmenter = iaa.Sequential([
#     iaa.Multiply((0.8, 1.2)),  
#     iaa.LinearContrast((0.8, 1.2)),  
# ])

# augmented_images = []

# for path in moderateDem:
#     path = os.path.join("/kaggle/input/better-preproc/train", path)
#     image = np.load(path)
    
#     for _ in range(30):
#         augmented_image = augmenter.augment_image(image)
#         augmented_images.append(augmented_image)
# augmented_images = np.array(augmented_images)

In [5]:
# len(augmented_images)

In [6]:
# import cv2
# import os


# output_directory = "/kaggle/working"

# os.makedirs(output_directory, exist_ok=True)

# for i, image in enumerate(augmented_images):
#     output_path = os.path.join(output_directory, f"moderateDem{i+53}.npy")
#     np.save(output_path, image)
    

# print("Augmented images saved successfully.")


In [7]:
import os
import numpy as np
directory = '/kaggle/input/betterer-preproc/archive/train'

files = [file for file in os.listdir(directory) if file.endswith('.npy')]
len(files)

6681

In [8]:
mildDem = [i for i in files if "mildDem" in i]
moderateDem = [i for i in files if "moderateDem" in i]
nonDem = [i for i in files if "nonDem" in i]
veryMild = [i for i in files if "verymildDem" in i]
print(len(mildDem), len(moderateDem), len(nonDem), len(veryMild))

2509 1612 2560 1792


In [9]:
import pandas as pd
l = [[mildDem, 0], [moderateDem, 1], [nonDem, 2], [veryMild, 3]]
x = []
for i in l:
    for j in i[0]:
        path = os.path.join("/kaggle/input/betterer-preproc/archive/train", j)
        x.append([i[1], path])
        
df = pd.DataFrame(x, columns=["Class", "Path"])
df["Class"].value_counts()
        

Class
2    2560
0    2509
3    1792
1    1612
Name: count, dtype: int64

In [10]:
images = []
for i in df["Path"]:
    images.append(np.load(i))
images = np.array(images)

In [11]:
from sklearn.model_selection import train_test_split
labels = np.array(df["Class"])
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)


In [12]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Verify the encoded labels
print(train_labels_encoded)
print(test_labels_encoded)

[3 2 1 ... 2 0 3]
[3 3 2 ... 3 0 1]


In [13]:
print(train_images.shape, test_images.shape, train_labels_encoded.shape, test_labels_encoded.shape)

(6778, 208, 176, 3) (1695, 208, 176, 3) (6778,) (1695,)


In [14]:
print(test_labels)
test_labels_encoded

[3 3 2 ... 3 0 1]


array([3, 3, 2, ..., 3, 0, 1])

In [15]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define CNN model
model = models.Sequential([
    layers.Input(shape=train_images[0].shape),  # Input layer with shape of input data
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(4)  # Number of classes
])

# Define precision and recall metrics
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()

# Define F1 score function
def f1_score(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))

# Compile the model
# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', tf.keras.metrics.SparseCategoricalAccuracy()])

# Train the model
history = model.fit(train_images, train_labels_encoded, epochs=10, validation_data=(test_images, test_labels_encoded))

2024-04-14 16:21:50.745342: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-14 16:21:50.745493: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-14 16:21:50.941837: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Epoch 1/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 251s 1s/step - accuracy: 0.4413 - loss: 40.8540 - sparse_categorical_accuracy: 0.4413 - val_accuracy: 0.5699 - val_loss: 0.8182 - val_sparse_categorical_accuracy: 0.5699
Epoch 2/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 262s 1s/step - accuracy: 0.6179 - loss: 0.7512 - sparse_categorical_accuracy: 0.6179 - val_accuracy: 0.6206 - val_loss: 0.7132 - val_sparse_categorical_accuracy: 0.6206
Epoch 3/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 265s 1s/step - accuracy: 0.6806 - loss: 0.6275 - sparse_categorical_accuracy: 0.6806 - val_accuracy: 0.6590 - val_loss: 0.6759 - val_sparse_categorical_accuracy: 0.6590
Epoch 4/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 261s 1s/step - accuracy: 0.7178 - loss: 0.5497 - sparse_categorical_accuracy: 0.7178 - val_accuracy: 0.6566 - val_loss: 0.6247 - val_sparse_categorical_accuracy: 0.6566
Epoch 5/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 263s 1s/step - accuracy: 0.7430 - loss: 0.4648 - sparse_categorical_accuracy: 0.7430 - val_accuracy: 0.6437 - val_loss: 0.6

In [16]:
import numpy as np
from sklearn.metrics import roc_auc_score

y_prob = model.predict(test_images)

y_prob = tf.nn.softmax(y_prob).numpy()

y_true = np.eye(4)[test_labels_encoded]

# Compute ROC AUC for each class
roc_auc = roc_auc_score(y_true, y_prob, average='macro')

print("ROC AUC:", roc_auc)


53/53 ━━━━━━━━━━━━━━━━━━━━ 17s 327ms/step
ROC AUC: 0.8826260024589454


In [17]:
# Save the model
model.save("my_cnn_model.h5")
